### This is old file, checkout the Traktv_2022.ipynb

In [ ]:
import csv
#import calplot
import pandas as pd
import matplotlib.pyplot as plt
from imdb import IMDb
from datetime import datetime

In [ ]:
from pyecharts.charts import Bar
from pyecharts.globals import CurrentConfig, NotebookType
from pyecharts import options as opts
from pyecharts.globals import ThemeType
from snapshot_selenium import snapshot
from pyecharts.render import make_snapshot

In [ ]:
import trakt.core
trakt.core.AUTH_METHOD = trakt.core.OAUTH_AUTH
trakt.core.init("mzfr", client_id="", client_secret="", store=True)


In [ ]:
from trakt.movies import Movie
from trakt.tv import TVShow

In [ ]:
# Download loads of information and might take sometime to run

movies_info = {}

with open("export_movies_history.csv", "r") as f:
    movies = csv.DictReader(f)
    for movie in movies:
        date_str = datetime.strptime(movie["watched_at"], "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%Y-%m-%d")
        date = datetime.strptime(date_str, "%Y-%m-%d")
        if date.year != 2022:
            continue
        else:
            code = movie["trakt"]

            if code in movies_info:
                movies_info[code]['count'] += 1
            else:
                movies_info[code] = {}
                movies_info[code]['count'] = 1
                movies_info[code]['date'] = date
                
                movie_details = Movie(code)
                movies_info[code]['runtime'] = movie_details.runtime
                movies_info[code]['genres'] = movie_details.genres


In [ ]:
# In this I took out information for all the years.
# If you are doing this then don't run the above cell because that
# was just for a year

detailed_info = {}

with open("export_movies_history.csv", "r") as f:
    movies = csv.DictReader(f)
    for movie in movies:
        date_str = datetime.strptime(movie["watched_at"], "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%Y-%m-%d")
        date = datetime.strptime(date_str, "%Y-%m-%d")
        code = movie["trakt"]

        if code in detailed_info:
            detailed_info[code]['count'] += 1
        else:
            detailed_info[code] = {}
            detailed_info[code]['count'] = 1
            detailed_info[code]['date'] = date
            
            movie_details = Movie(code)
            detailed_info[code]['runtime'] = movie_details.runtime
            detailed_info[code]['genres'] = movie_details.genres

In [ ]:
# Trying to compare genre's watched from two years

yearly_genre = {"2022":{}, "2021": {}}
for _, movie in detailed_info.items():
    yr = str(movie['date'].year)
    if yr not in ['2022', '2021']:
        continue
    for g in movie['genres']:
        if g in yearly_genre[yr]:
            yearly_genre[yr][g] += 1
        else:
            yearly_genre[yr][g] = 1


In [ ]:
# Merging the information obtained from previous cell
from collections import defaultdict
dd = defaultdict(list)

for d in (yearly_genre['2022'], yearly_genre['2021']):
    for key, value in d.items():
        dd[key].append(value)

In [ ]:
# This cell take information from the movies_info cell and gives you the count of genre of a specific year only
# Good for Single year chart
totalTime = 0
genres = {}
for code, info in movies_info.items():
    if info['date'].year != 2022:
        continue
    totalTime += int(info['runtime'])
    for g in info["genres"]:
        if g in genres:
            genres[g] += 1
        else:
            genres[g] = 1    

In [ ]:
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
data = dict(sorted(genres.items(), key=lambda item: item[1], reverse=True))
bar = Bar(init_opts=opts.InitOpts(theme=ThemeType.VINTAGE))

vals = list(data.values())
genre = list(data.keys())
bar.add_yaxis("No.Movies", vals)
bar.add_xaxis(genre)
bar.reversal_axis()
bar.set_series_opts(label_opts=opts.LabelOpts(position=None))
# bar.render()
make_snapshot(snapshot, bar.render(), "movies_genre_2022.png")


In [ ]:
# Use Pyechart instead
# This cell depends on pandas/matplotlib so maybe don't use it :)

print("TotalTime spent watching movies: ", totalTime//60, "hr")

data = dict(sorted(genres.items(), key=lambda item: item[1], reverse=True))
genre_df = pd.DataFrame(data.items(), columns=["Genre", "No. of Movies"])
genre_df.plot(kind="barh", x=0,y=1, figsize=(10,5), color='blue')


In [ ]:
tvShows = {}
dayWiseTV = {}
with open("export_shows_history.csv", "r") as f:
    shows = csv.DictReader(f)
    for show in shows:
        date_str = datetime.strptime(show["watched_at"], "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%Y-%m-%d")
        date = datetime.strptime(date_str, "%Y-%m-%d")
        code = show["trakt"]
        if date.year != 2021:
            continue
        else:
            if code in tvShows:
                tvShows[code]["count"] += 1
            else:
                tvShows[code] = {}
                tvShows[code]["count"] = 1
                tvShows[code]["date"] = date_str
#                 imdb_info = ia.get_movie(code)
                # Some indian shows didn't had these values
                show = TVShow(code)
                tvShows[code]["runtime"] = show.runtime
                tvShows[code]["genres"] = show.genres

            if date in dayWiseTV:
                dayWiseTV[date] += 1
            else:
                dayWiseTV[date] = 1

In [ ]:
totalTvTime = 0
tvGenres = {}
for code, info in tvShows.items():
    if "2021" not in info['date']:
        continue
    if "runtime" in info:
        # Even on imdb the data is bit uneven
        # For some shows the runtime tells you the complete runtime of the show
        # Ex: The Queen's gambit there are 7 episode and total time of 6h 33 min
        #     So runtime for that was 393 instead of 1 episode
        # That is why here if any show have the runtime of more than an hour that
        #  means it the total time and not the time of an episode
        if int(info["runtime"]) > 60:
            totalTvTime += int(info["runtime"])
        else:
            totalTvTime += (int(info['runtime'])*info["count"])
    if "genres" in info:
        for g in info["genres"]:
            if g in tvGenres:
                tvGenres[g] += 1
            else:
                tvGenres[g] = 1  

In [ ]:
print("TotalTime spent watching TV Shows: ", totalTvTime//60, "hr")

data = dict(sorted(tvGenres.items(), key=lambda item: item[1], reverse=True))
tvgenre_df = pd.DataFrame(data.items(), columns=["Genre", "Count"])
tvgenre_df.plot(kind="barh", x=0,y=1, figsize=(10,5), color='blue')

In [ ]:
dataToPlot = {}

for k,v in dayWiseTV.items():
    if k.year == 2021:
        dataToPlot[k] = v

series = pd.Series(dataToPlot)
calplot.calplot(series, cmap='YlGn', edgecolor=None, yearlabels=False)